# Information retrieval --- Module 1: Evaulation
Santosh Kumar Rajamanickam, Dennis Ulmer and Stian Steinbakken

Maybe insert a small introduction here? 


## Theoretical part [15pts]
### 1 Hypothesis Testing --- The problem of multiple comparisons. 
How many hypothesis tests, m, does it take to get to (with Type I error for each test = α):
P(mth experiment gives significant result | m experiments lacking power to reject H0)?
P(at least one significant result | m experiments lacking power to reject H0)?



Answer:

### 2 Bias and unfairness in Interleaving experiments [10 points]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?


Answer:

## Experimental part [85 pts] 
Step 1: Simulate Rankings of Relevance for E and P (5 points)
In the first step you will generate pairs of rankings of relevance, for the production P and experimental E, respectively, for a hypothetical query q. Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all possible P and E ranking pairs of length 5. This step should give you about.

Example:
P: {N N N N N}
E: {N N N N R}
…
P: {HR HR HR HR R}
E: {HR HR HR HR HR}


In [5]:
# Step 1 code

from itertools import product
from collections import namedtuple

RankingPair = namedtuple("RankingPair", ["E", "P"])

RANKINGS = ("N", "R", "HR")

simulations = []

# Create all possible rankings of length 5
for ranking_pair in list(product((list(product(RANKINGS, repeat=5)), list(product(RANKINGS, repeat=5))), repeat=2)):
    print(ranking_pair)
    simulations.append(RankingPair(E=ranking_a, P=ranking_b))

print("First 10 simulations:\n")
for ranking_pair in simulations[:10]:
    print("P: {}, E: {}".format(*ranking_pair))
    
print("\nLast 10 simulations:\n")
for P_ranking, E_ranking in simulations[-10:]:
    print("P: {}, E: {}".format(*ranking_pair))


([('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'R'), ('N', 'N', 'N', 'N', 'HR'), ('N', 'N', 'N', 'R', 'N'), ('N', 'N', 'N', 'R', 'R'), ('N', 'N', 'N', 'R', 'HR'), ('N', 'N', 'N', 'HR', 'N'), ('N', 'N', 'N', 'HR', 'R'), ('N', 'N', 'N', 'HR', 'HR'), ('N', 'N', 'R', 'N', 'N'), ('N', 'N', 'R', 'N', 'R'), ('N', 'N', 'R', 'N', 'HR'), ('N', 'N', 'R', 'R', 'N'), ('N', 'N', 'R', 'R', 'R'), ('N', 'N', 'R', 'R', 'HR'), ('N', 'N', 'R', 'HR', 'N'), ('N', 'N', 'R', 'HR', 'R'), ('N', 'N', 'R', 'HR', 'HR'), ('N', 'N', 'HR', 'N', 'N'), ('N', 'N', 'HR', 'N', 'R'), ('N', 'N', 'HR', 'N', 'HR'), ('N', 'N', 'HR', 'R', 'N'), ('N', 'N', 'HR', 'R', 'R'), ('N', 'N', 'HR', 'R', 'HR'), ('N', 'N', 'HR', 'HR', 'N'), ('N', 'N', 'HR', 'HR', 'R'), ('N', 'N', 'HR', 'HR', 'HR'), ('N', 'R', 'N', 'N', 'N'), ('N', 'R', 'N', 'N', 'R'), ('N', 'R', 'N', 'N', 'HR'), ('N', 'R', 'N', 'R', 'N'), ('N', 'R', 'N', 'R', 'R'), ('N', 'R', 'N', 'R', 'HR'), ('N', 'R', 'N', 'HR', 'N'), ('N', 'R', 'N', 'HR', 'R'), ('N', 'R', 'N', 'HR', 

NameError: name 'ranking_a' is not defined